# Data Processing Pipeline
*Client: Royal Perth Hospital*

*Team: Group 7*

# Readme
The following libraries need to be installed in order to run the source code.

In [15]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from importlib import reload
# import utility functions
import src.utils as utils

# Note: to run main notebook from root directory, use:
ROOT_DIR = Path('')
# setup OS agnostic pathnames
# ROOT_DIR = Path('..')

import src.DataLoader as DataLoader
reload(DataLoader)

dataLoader = DataLoader.DataLoader(ROOT_DIR)

# 1.Dataset Processing

Load the dataset. The following tables are loaded for this project:
- LABEVENTS
- MICROBIOLOGYEVENTS
- D_ICD_DIAGNOSES
- D_ITEMS

In [2]:
df_desc_icd, df_desc_labitems, df_desc_items = dataLoader.load_descriptions()

The DIAGNOSES_ICD tables has a column ICD9_CODE which is the code for each disease diagnosed for the patient.

- The Sepsis has 6 codes: ['77181', '99591', '99592', '67020', '67022', '67024']

- We'll introduce a new column, IS_SEPSIS, as a binary classifier (1 for 6 sepsis ICD9 codes, 0 otherwise) for the target variable.

In [3]:
df_diagnoses_icd = dataLoader.load_diagnoses_icd(df_desc_icd)

The Demographic dataframe is a combination of the patient and admission datasets. Adding some columns:
- AGE: the age of the patient at the admitted year
- IS_SEPSIS: indicating whether this is a sepsis case or not


The labevents dataframe is loaded. Admissions under 18 years old are removed

In [4]:
df_demographic = dataLoader.load_demographic(df_diagnoses_icd)

Removed 8210 admissions with AGE < 18


We replace those with aged > 100 by the median age of patients

In [5]:
df_demographic = dataLoader.demographic_clean_AGE(df_demographic)

The labevents dataframe is loaded. The records of admissions under 18 years old are removed.
- The empty HADM_ID is this are cleaned

In [6]:
df_labevents = dataLoader.load_labevents(df_demographic)

Calculate the TIME from for labevents from ADMITTIME to CHARTTIME
- A column NEW_ADMITTIME is added: if CHARTTIME < ADMITTIME, the NEW_ADMITTIME is the CHARTTIME, otherwise, use the ADMITTIME

In [9]:
df_labevents = dataLoader.labevents_compute_TIME(df_labevents, df_demographic)

# 2. Create train data

In [16]:
dataLoader.create_train_data(df_labevents, df_demographic, df_desc_labitems, hours=0, feature_no=20, output_filename=ROOT_DIR/"data/Model input data/test_new_file.csv")

In [9]:
dataLoader.create_train_data(df_labevents, df_demographic, df_desc_labitems, hours=2, feature_no=20)

In [10]:
dataLoader.create_train_data(df_labevents, df_demographic, df_desc_labitems, hours=4, feature_no=20)

In [70]:
df_final = dataLoader.create_train_data_sequence(df_labevents, df_demographic, df_desc_labitems, 4)

In [18]:
df_final = dataLoader.create_train_data_sequence(df_labevents, df_demographic, df_desc_labitems, 8, output_filename = ROOT_DIR / 'data/Model input data/t8_sequence.csv')

In [12]:
df_final = dataLoader.extract_train_data_by_features(df_labevents, df_demographic, df_desc_labitems, 4, ROOT_DIR / 'data/potential_labevents_count_with_proportion.csv', 20)

In [17]:
itemID = ["ITEMID_51222", "ITEMID_51221", "ITEMID_51279", "ITEMID_50931", "ITEMID_51301", "ITEMID_51256", "ITEMID_51274", 
          "ITEMID_50970", "ITEMID_51006", "ITEMID_50893", "ITEMID_51244", "ITEMID_50902", "ITEMID_50882", "ITEMID_50821", 
          "ITEMID_51265", "ITEMID_51248", "ITEMID_51275", "ITEMID_51277", "ITEMID_50820", "ITEMID_50818", "ITEMID_50885", "ITEMID_50912"]
dataLoader.convert_itemid_to_title(itemID, df_desc_labitems)

{'ITEMID_51222': 'Hemoglobin (51222)',
 'ITEMID_51221': 'Hematocrit (51221)',
 'ITEMID_51279': 'Red Blood Cells (51279)',
 'ITEMID_50931': 'Glucose (50931)',
 'ITEMID_51301': 'White Blood Cells (51301)',
 'ITEMID_51256': 'Neutrophils (51256)',
 'ITEMID_51274': 'PT (51274)',
 'ITEMID_50970': 'Phosphate (50970)',
 'ITEMID_51006': 'Urea Nitrogen (51006)',
 'ITEMID_50893': 'Calcium, Total (50893)',
 'ITEMID_51244': 'Lymphocytes (51244)',
 'ITEMID_50902': 'Chloride (50902)',
 'ITEMID_50882': 'Bicarbonate (50882)',
 'ITEMID_50821': 'pO2 (50821)',
 'ITEMID_51265': 'Platelet Count (51265)',
 'ITEMID_51248': 'MCH (51248)',
 'ITEMID_51275': 'PTT (51275)',
 'ITEMID_51277': 'RDW (51277)',
 'ITEMID_50820': 'pH (50820)',
 'ITEMID_50818': 'pCO2 (50818)',
 'ITEMID_50885': 'Bilirubin, Total (50885)',
 'ITEMID_50912': 'Creatinine (50912)'}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Sample data (replace with your actual data)
sepsis_data = df_demographic

microbiology_data = dataLoader.load_microbiologyevents()
microbiology_data
# Filter the microbiology data to include only rows with sepsis patients
sepsis_hadm_ids = sepsis_data['HADM_ID'].tolist()
valid_spec_types = ['URINE', 'BLOOD CULTURE', 'STOOL']

filtered_microbiology_data = microbiology_data[microbiology_data['HADM_ID'].isin(sepsis_hadm_ids)]

filtered_microbiology_data = filtered_microbiology_data[filtered_microbiology_data['SPEC_TYPE_DESC'].isin(valid_spec_types)]
# Group the filtered microbiology data by bacteria and count occurrences
bacteria_counts = filtered_microbiology_data['ORG_NAME'].value_counts()

# Sort the bacteria by frequency in descending order
bacteria_counts = bacteria_counts.sort_values(ascending=False)
top_30_bacteria_counts = bacteria_counts.head(30)
# Create a bar chart to show the most common bacteria affecting sepsis patients
plt.figure(figsize=(8, 10))  # Adjust the figure size as needed
top_30_bacteria_counts = top_30_bacteria_counts.iloc[::-1]
plt.barh(top_30_bacteria_counts.index, top_30_bacteria_counts.values, color='skyblue')
plt.title('Top 30 Most Common Bacteria Affecting Sepsis Patients')
plt.xlabel('Frequency')
plt.ylabel('Bacteria')

# Show the bar chart
plt.tight_layout()
plt.show()
# In this updated code, we've changed the plt.bar to plt.barh to create a horizontal bar chart, which effectively rotates the plot 90 degrees to the right. We've also adjusted the figure size to ensure the plot looks appropriate in the landscape orientation.

# Show the bar chart
plt.show()

In [ ]:
#06/10/2023 start  CHECKING the importance of Microbiology events
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
# Sample data (replace with your actual data)
sepsis_data = df_demographic
df_micro = pd.DataFrame(dataLoader.load_microbiologyevents())

# Convert CHARTTIME column to datetime
df_micro['CHARTTIME'] = pd.to_datetime(df_micro['CHARTTIME'], errors='coerce', format='%d/%m/%Y %H:%M')


# Merge the two dataframes based on HADM_ID
df_micro_demo = pd.merge(df_micro, sepsis_data, on='HADM_ID')


selected_columns = ['HADM_ID', 'CHARTTIME', 'IS_SEPSIS', 'ORG_NAME']
filtered_df = df_micro_demo[selected_columns]
filtered_df

import pandas as pd

merged_df_sorted = filtered_df.sort_values(by='HADM_ID')


# Define conditions for filtering the DataFrame
condition1 = (merged_df_sorted['IS_SEPSIS'] == 1) & (~merged_df_sorted['ORG_NAME'].isnull())
condition2 = (merged_df_sorted['IS_SEPSIS'] == 0) & (merged_df_sorted['ORG_NAME'].isnull())


merged_df_sorted['Flag'] = np.where(condition1 | condition2, 'Pass', 'Fail')


merged_df_sorted.to_excel('filtered_and_flagged_data.xlsx', index=False)


In [ ]:
# Separate data into 'Pass' and 'Fail' DataFrames
pass_df = merged_df_sorted[merged_df_sorted['Flag'] == 'Pass']
fail_df = merged_df_sorted[merged_df_sorted['Flag'] == 'Fail']

# Get unique rows for each HADM_ID in 'Pass' DataFrame
unique_pass_df = pass_df.drop_duplicates(subset='HADM_ID')

# Get unique rows for each HADM_ID in 'Fail' DataFrame
unique_fail_df = fail_df.drop_duplicates(subset='HADM_ID')

# Export 'Pass' and 'Fail' DataFrames to Excel files
pass_df.to_excel('pass_data.xlsx', index=False)
fail_df.to_excel('fail_data.xlsx', index=False)

# Export unique rows for 'Pass' and 'Fail' HADM_ID to Excel files
unique_pass_df.to_excel('unique_pass_data.xlsx', index=False)
unique_fail_df.to_excel('unique_fail_data.xlsx', index=False)

In [ ]:
# Concatenate unique_pass_df and unique_fail_df without overlapping HADM_ID

filtered_fail_df = unique_fail_df[~unique_fail_df['HADM_ID'].isin(unique_pass_df['HADM_ID'])]

# Concatenate pass_df and filtered_fail_df into whole_df
whole_df = pd.concat([unique_pass_df, filtered_fail_df], ignore_index=True)

# Export the concatenated DataFrame to an Excel file
whole_df.to_excel('whole_data_pass_and_fail.xlsx', index=False)

In [ ]:
# Count the occurrences of 'Pass' and 'Fail' in the 'Flag' column
flag_counts = whole_df['Flag'].value_counts()

# Calculate proportions
pass_proportion = flag_counts.get('Pass', 0) / len(whole_df)
fail_proportion = flag_counts.get('Fail', 0) / len(whole_df)

# Print proportions
print(f'Proportion of Flag = Pass: {pass_proportion:.2f}')
print(f'Proportion of Flag = Fail: {fail_proportion:.2f}')

labels = ['Pass', 'Fail']
sizes = [pass_proportion, fail_proportion]  # Replace with your actual proportions
colors = ['lightblue', 'lightcoral']
explode = (0.1, 0)  # Explode the "Pass" slice (optional)


plt.figure(figsize=(3, 3))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that the pie is drawn as a circle.

plt.title('Proportion of Pass and Fail of Theory')

plt.show()

#06/10/2023 end